In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [3]:

llm = ChatOpenAI(
        model="gpt-4o-mini",
        #temperature=0,
        #max_tokens=None,
        #timeout=None,
        #max_retries=2,
        # api_key="...",  # if you prefer to pass api key in directly instaed of using env vars
        # base_url="...",
        # organization="...",
        # other params...
    )


In [16]:

loader = PyPDFLoader("CleanCode.pdf")
data = loader.load()


In [17]:
text = "\n".join([doc.page_content for doc in data])
print(len(text))

895495


In [29]:

##Text splitter
text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=256,
       chunk_overlap=50,
       length_function=len
    )
chunks = text_splitter.split_text(text)
for i,chunk in enumerate(chunks[:3]):
    print(f"   Chunk {i+1}: {chunk}")

   Chunk 1: Clean Code
Robert C. Martin Series
The mission of this series is to improve the state of the art of software craftsmanship. 
The books in this series are technical, pragmatic, and substantial. The authors are
   Chunk 2: highly experienced craftsmen and professionals dedicated to writing about what 
actually works in practice, as opposed to what might work in theory. You will read 
about what the author has done, not what he thinks you should do. If the book is
   Chunk 3: about programming, there will be lots of code. If the book is about managing, there 
will be lots of case studies from real projects. 
These are the books that all serious practitioners will have on their bookshelves.


In [30]:


emb_llm = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [31]:

##vector db
vectordb = Chroma.from_texts(chunks, emb_llm)

In [46]:

##retriver
retriever = vectordb.as_retriever(search_kwargs={'k': 10})

In [47]:
query = "What is Clean Code"
### QA Chain

from_vdb =  retriever.invoke(query);
from_vdb

[Document(metadata={}, page_content='The Art of Clean Code?\nLet’ s say you believe that messy code is a signiﬁcant impediment. Let’ s say that you accept\nthat the only way to go fast is to keep your code clean. Then you must ask yourself: “How'),
 Document(metadata={}, page_content='detail, what I and my compatriots think about\nclean code. We will tell you what we think makes\na clean variable name, a clean function, a clean\nclass, etc. We will present these opinions as abso-\nlutes, and we will not apologize for our stridence.'),
 Document(metadata={}, page_content='Indeed, that value system has been the goal, and the topic, of this book. Clean code is\nnot written by following a set of rules. Y ou don’t become a software craftsman by learn-'),
 Document(metadata={}, page_content='That’ s what makes clean code for me.\nHere, in a few short paragraphs, Ron has summarized the contents of this book. No\nduplication, one thing, expressiveness, tiny abstractions. Everything is there. \

In [48]:

qa = RetrievalQA.from_chain_type(llm=llm,
                                     chain_type="stuff",
                                     retriever=retriever,
                                     return_source_documents=False)
response = qa.invoke(query)
response['result']


# Create Gradio interface
# rag_application = gr.Interface(
#     fn=retriever_qa,
#     flagging_mode="never",
#     inputs=[
#         gr.File(label="Upload PDF File", file_count="single", file_types=['.pdf'], type="filepath"),
#         # Drag and drop file upload
#         gr.Textbox(label="Input Query", lines=2, placeholder="Type your question here...")
#     ],
#     outputs=gr.Textbox(label="Output"),
#     title="RAG Chatbot",
#     description="Upload a PDF document and ask any question. The chatbot will try to answer using the provided document."
# )
#
# rag_application.launch(server_name="127.0.0.1", server_port=7860)



'The text does not provide specific principles of clean code but emphasizes that clean code has certain qualities such as being readable, maintainable, and organized. It mentions characteristics like meaningful names, minimal dependencies, and a focus on expressiveness and simplicity. It also highlights the idea that writing clean code requires care and discipline. For a comprehensive list of principles, you may need to refer to additional resources or literature on the topic.'